In [ ]:
from ase.build import bulk
from ase.calculators.espresso import Espresso
from ase.io import write
from pwtools import io

In [ ]:
import subprocess

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

# Quantum Espresso Inputs

In [ ]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [ ]:
# https://www.quantum-espresso.org/Doc/INPUT_PW.html
input_data_relax = {
    'calculation': 'vc-relax', # A string describing the task to be performed.
    'ecutwfc': 29.49,  # kinetic energy cutoff (Ry) for wavefunctions
    'conv_thr': 1e-06,  # Convergence threshold for selfconsistency
    'diagonalization': 'david',  
    'electron_maxstep': 100,  # maximum number of iterations in a scf step. 
    'nstep': 200,  # number of molecular-dynamics or structural optimization steps performed in this run.
    'etot_conv_thr': 1e-4,  # Convergence threshold on total energy (a.u) for ionic minimization
    'forc_conv_thr': 1e-3,  # Convergence threshold on forces (a.u) for ionic minimization
    'smearing': 'methfessel-paxton',  # ordinary Gaussian spreading (Default)
}

In [ ]:
input_data_static = {
    'calculation': 'scf', # A string describing the task to be performed.
    'ecutwfc': 29.49,  # kinetic energy cutoff (Ry) for wavefunctions
    'conv_thr': 1e-06,  # Convergence threshold for selfconsistency
    'diagonalization': 'david',  
    'electron_maxstep': 100,  # maximum number of iterations in a scf step. 
    'smearing': 'methfessel-paxton',  # ordinary Gaussian spreading (Default)
}

# Workflow

## Structure Optimization

In [ ]:
structure = bulk('Al', a=4.15, cubic=True)

In [ ]:
write(
    'relax.pwi', 
    structure, 
    Crystal=True, 
    kpts=(3, 3, 3), 
    input_data=input_data_relax, 
    pseudopotentials=pseudopotentials,
    tstress=True, 
    tprnfor=True
)

In [ ]:
subprocess.check_output("mpirun -np 4 pw.x -in relax.pwi > relax.pwo", shell=True, universal_newlines=True)

In [ ]:
structure_opt = io.read_pw_md('relax.pwo')[-1].get_ase_atoms()
structure_opt

## Energy Volume Curve

In [ ]:
energy_lst, volume_lst = [], []
for strain in np.linspace(0.9, 1.1, 5):
    structure_strain = structure_opt.copy()
    structure_strain = structure.copy()
    structure_strain.set_cell(structure_strain.cell * strain**(1/3), scale_atoms=True)
    write(
        'strain.pwi', 
        structure_strain, 
        Crystal=True, 
        kpts=(3, 3, 3), 
        input_data=input_data_static, 
        pseudopotentials=pseudopotentials,
        tstress=True, 
        tprnfor=True
    )
    subprocess.check_output("mpirun -np 4 pw.x -in strain.pwi > strain.pwo", shell=True, universal_newlines=True)
    out = io.read_pw_scf('strain.pwo')
    energy_lst.append(out.etot)
    volume_lst.append(out.volume)

# Result

In [ ]:
plt.plot(volume_lst, energy_lst)
plt.xlabel("Volume")
plt.ylabel("Energy")